In [1]:
import config

import pandas as pd
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
train = pd.read_pickle("./data/train.pkl")
test = pd.read_pickle("./data/test.pkl")
X_train, y_train = train["problem_statement"], train["tags"]
X_test, y_test = test["problem_statement"], test["tags"]

print(X_train.tail())
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

464E     shortest number path weight given state undire...
25C      known shortest open guarante number row given ...
107B     mani guarante correspond wafa power point find...
1181A    mani negat find per solut economi changa pay p...
1081B    weird parti poor find wore similar forgot solu...
Name: problem_statement, dtype: object
(6272,) (6272,)
(1568,) (1568,)


In [3]:
# multilabel binarizer
mlb = MultiLabelBinarizer()
y_train_onehot = pd.DataFrame(mlb.fit_transform(y_train),
    columns=mlb.classes_, index=y_train.index)
y_test_onehot = pd.DataFrame(mlb.fit_transform(y_test),
    columns=mlb.classes_, index=y_test.index)

print(mlb.classes_)
print(y_train_onehot.shape)
print(X_train.shape, y_train_onehot["implementation"].shape)


['2-sat' 'binarysearch' 'bitmasks' 'bruteforce' 'chineseremaindertheorem'
 'combinatorics' 'constructivealgorithms' 'datastructures' 'dfsandsimilar'
 'divideandconquer' 'dp' 'dsu' 'expressionparsing' 'fft' 'flows' 'games'
 'geometry' 'graphmatchings' 'graphs' 'greedy' 'hashing' 'implementation'
 'interactive' 'math' 'matrices' 'meet-in-the-middle' 'nan' 'numbertheory'
 'probabilities' 'schedules' 'shortestpaths' 'sortings' 'strings'
 'stringsuffixstructures' 'ternarysearch' 'trees' 'twopointers']
(6272, 37)
(6272,) (6272,)


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)
X_test_counts = count_vect.transform(X_test)
print(X_test_counts.shape)


(6272, 12569)
(1568, 12569)


In [5]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

tags = mlb.classes_
for tag in tags:
    clf.fit(X_train_counts, y_train_onehot[tag])

def predict(ii):
    X_test_ii = X_test_counts[ii]
    y_preds = []
    for tag in tags:
        clf.fit(X_train_counts, y_train_onehot[tag])
        y_pred = clf.predict(X_test_ii)
        y_preds.extend(y_pred)
    # print(y_preds)
    # print(list(y_test_onehot.iloc[ii]))
    # print(y_test[ii])

    y_ans = []
    for i in range(len(y_preds)):
        if y_preds[i]: 
            y_ans.append(tags[i])
    return y_ans

print(predict(1000), y_test.iloc[1000])

['greedy', 'math'] ['datastructures', 'greedy']
